# ヒトTTN遺伝子のイントロンの5'、3'側末端の塩基の保存性の評価

**目的**
イントロン同士の配列を比較して、相同性を確認する<br>
相同性とスプライシングパターンの関連がないかを検証

**注意点**
ストランドの向きを注意

In [10]:
# オリジナルモジュールのインポート
from lib.gbkparse import Seq_count
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import subprocess
import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 可視化の関数の定義

In [17]:


def fig_show(gene, num=80):
    gbk_file = f"../data/gbk/human_{gene}.gb"
    ucsc_file = f"../data/ucsc/ucsc_{gene}.txt"

    # クラスのインスタンス化
    gbk = Seq_count()

    # gbkファイルの読み込み
    gbk.read_gbk(gbk_file)
    with open(gbk_file) as f:
        lines = f.readlines()
        for i in lines[:10]:
            if "ACCESSION" in i:
                if "complement" in i:
                    strand = "-"
                    start = int(i.split(" ")[-1].replace("complement(", "").replace(")","").split("..")[0])
                    end = int(i.split(" ")[-1].replace("complement(", "").replace(")","").split("..")[1])
                else:
                    strand = "+"
                    start = int(i.split(" ")[-1].split("..")[0])
                    end = int(i.split(" ")[-1].split("..")[1])
    # print(strand, start, end)

    ## UCSCのTbale BrowserからダウンロードしたphastCons100wayのデータの読み込み

    # ucscのconservationデータの読み込み
    df = pd.read_csv(ucsc_file, sep='\t', skiprows=9, header=None)  

    # 遺伝子範囲のみ抽出
    df = df[(df[0] >= start) & (df[0] <= end)]
    df.columns = ['pos','cons' ]

    # マイナスストランドの場合の処理
    if strand == "-":
        df['pos'] = df['pos'].values[::-1]
        df['cons'] = df['cons'].values[::-1]
        df['pos'] = df['pos'].max() - df['pos']
    else:
        df['pos'] = df['pos'] - df['pos'].min()

    # 欠損値の処理
    df = df.set_index('pos', drop=True)
    tmp = pd.DataFrame([i for i in range(len(df.index))])
    df = pd.merge(tmp, df, left_on=0, right_on='pos', how='outer')
    df = df.iloc[:,1].to_frame()
    ## 各イントロンとエクソンの塩基ごとの保存性の平均値を求める
    # エクソンのごとの保存性の平均値を求める
    exon_cons_ave = {}
    for i in range(gbk.exon_num()):
        r = gbk.exon_list()[i]
        ave = round(df.iloc[r[0]:r[1], 0].mean(),2)
        exon_cons_ave[i+1] = ave

    # イントロンのごとの保存性の平均値を求める
    intron_cons_ave = {}
    for i in range(gbk.intron_num()):
        r = gbk.intron_list()[i]
        ave = round(df.iloc[r[0]:r[1], 0].mean(),2)
        intron_cons_ave[i+1] = ave
    # 上記のデータの可視化
    exon_hovertxt = 'Exon: %{x}<br>Conservation: %{y}<extra></extra>'
    intron_hovertxt = 'Intron: %{x}<br>Conservation: %{y}<extra></extra>'
    fig = make_subplots(rows=2, cols=1, subplot_titles=("各エクソンの塩基の保存性の平均", "各イントロンの塩基の保存性の平均"))
    fig.add_trace(go.Bar(x=list(exon_cons_ave.keys()), y=list(exon_cons_ave.values()), hovertemplate=exon_hovertxt), row=1, col=1)
    fig.add_trace(go.Bar(x=list(intron_cons_ave.keys()), y=list(intron_cons_ave.values()), hovertemplate=intron_hovertxt), row=2, col=1)
    fig.update_yaxes(range=[0, 1], row=1, col=1)
    fig.update_yaxes(range=[0, 1], row=2, col=1)
    fig.update_layout(height=500, width=1200, title_text=f"{gene.upper()}塩基の保存性の平均値")
    fig.show()
    ## 各イントロン末端からnum塩基の各塩基の保存性を可視化
    # 検討するイントロン末端からの塩基数
    num = num

    # 5'側のイントロンn塩基の保存性の平均
    left_intron_cons = {} 
    for n, i in enumerate(gbk.intron_list()):
        left_intron_cons[n+1] = list(df.iloc[i[0]:i[0]+num,0])
    left_intron_cons_ave_df = pd.DataFrame.from_dict(left_intron_cons).T
    left_intron_cons_ave_df.columns = [i for i in range(1, num+1)]
    left_intron_cons_ave_df = left_intron_cons_ave_df.apply(lambda x: round(x,2))
    # 5'側のイントロンn塩基の保存性の平均
    right_intron_cons = {} 
    for n, i in enumerate(gbk.intron_list()):
        right_intron_cons[n+1] = list(df.iloc[i[1]-num:i[1],0])
    right_intron_cons_ave_df = pd.DataFrame.from_dict(right_intron_cons).T
    right_intron_cons_ave_df.columns = [-i for i in range(num, 0, -1)]
    right_intron_cons_ave_df = right_intron_cons_ave_df.apply(lambda x: round(x,2))
    # 上記データの可視化
    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"各イントロンの5'側の{num}塩基の保存性", f"各イントロンの3'側の{num}塩基の保存性"))
    left_hovertxt = 'Intron: %{y}<br>Base: %{x}<br>Conservation: %{z}<extra></extra>'
    right_hovertxt = 'Intron: %{y}<br>Base: %{x}<br>Conservation: %{z}<extra></extra>'
    fig.add_trace(go.Heatmap(x=left_intron_cons_ave_df.columns, y=left_intron_cons_ave_df.index, z=left_intron_cons_ave_df.values, colorscale='Viridis', hovertemplate=left_hovertxt), row=1, col=1)
    fig.add_trace(go.Heatmap(x=right_intron_cons_ave_df.columns, y=right_intron_cons_ave_df.index, z=right_intron_cons_ave_df.values, colorscale='Viridis', hovertemplate=right_hovertxt), row=1, col=2)
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(height=750, width=1200, title_text=f"{gene.upper()}の各イントロン両末端の保存性")
    fig.show()

In [22]:
fig_show('ldb3',80)

デフォルト値として、最もエクソンの多いNM_001368066.1を設定


In [21]:
fig_show('tnnt2',80)

デフォルト値として、最もエクソンの多いNM_001276345.2を設定
